In [2]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor

/Users/mariofilho/miniconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [3]:
data = pd.read_feather("train.f")
test = pd.read_feather("test.f")

# Retrain test score

In [8]:
%%time
X_train = data.filter(regex=r'feature')
y_train = data['target_kazutsugi']

mdl = LGBMRegressor(max_depth=5, num_leaves=2**5, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1, random_state=0)
mdl.fit(X_train, y_train)


CPU times: user 9min 6s, sys: 13.5 s, total: 9min 19s
Wall time: 55 s


LGBMRegressor(colsample_bytree=0.1, learning_rate=0.01, max_depth=5,
              n_estimators=2000, num_leaves=32, random_state=0)

In [10]:
results = []
for era in test['era'].unique():
    subset_test = test[test['era'] == era]
    
    X_test = subset_test.filter(regex=r'feature')
    y_test = subset_test['target_kazutsugi']
    
    predictions = pd.Series(mdl.predict(X_test))
    ranked_predictions = predictions.rank(pct=True, method="first")
    correlation = np.corrcoef(y_test, ranked_predictions)[0, 1]
    print("Era {} - Correlation {}".format(era, correlation))
    
    results.append(correlation)

Era 121 - Correlation 0.01988833510112423
Era 122 - Correlation 0.04441003942676335
Era 123 - Correlation 0.051055688096434675
Era 124 - Correlation 0.06696999757627116
Era 125 - Correlation 0.03853310254346967
Era 126 - Correlation 0.02267351143792772
Era 127 - Correlation -0.0034682507687966864
Era 128 - Correlation 0.06716680933507646
Era 129 - Correlation 0.0026536863309964847
Era 130 - Correlation 0.060540059305494984
Era 131 - Correlation 0.04461245928162007
Era 132 - Correlation 0.08144293241593094
Era 197 - Correlation 0.03267673531232786
Era 198 - Correlation 0.02868276105789081
Era 199 - Correlation -0.044331622497900604
Era 200 - Correlation 0.013917958105140179
Era 201 - Correlation 0.0013320679081422788
Era 202 - Correlation 0.04402592291484105
Era 203 - Correlation 0.02578797773477427
Era 204 - Correlation 0.05396320025156498
Era 205 - Correlation -0.007317445794865542
Era 206 - Correlation -0.0024665864618829253


In [11]:
np.median(results)

0.030679748185109337

In [12]:
np.min(results)

-0.044331622497900604

In [13]:
np.max(results)

0.08144293241593094

In [14]:
np.mean(results)

0.029215879027833888

In [15]:
len(results)

22

In [23]:
(np.array(results) < 0.00677).sum()

3

In [ ]:
(np.array(results) > 0.06416).sum()